In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, auc, roc_curve
from collections import Counter

# 1. Supervised, Semi-Supervised, and Unsupervised Learning

## (a) Download the Breast Cancer Wisconsin (Diagnostic) Data Set from: https://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+ %28Diagnostic%29. Download the data in https://archive.ics.uci.edu/ml/ machine-learning-databases/breast-cancer-wisconsin/wdbc.data, which has IDs, classes (Benign=B, Malignant=M), and 30 attributes. This data has two output classes.

In [2]:
df = pd.read_csv("../data/wdbc.data", header = None, index_col = False)
df = df.iloc[:,1:]
df.head(10)

,1,2,3,4,5,6,7,8,9,10,...,22,23,24,25,26,27,28,29,30,31
0,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.30010,0.14710,0.2419,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.08690,0.07017,0.1812,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.19740,0.12790,0.2069,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.24140,0.10520,0.2597,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.19800,0.10430,0.1809,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678
5,M,12.45,15.70,82.57,477.1,0.12780,0.17000,0.15780,0.08089,0.2087,...,15.47,23.75,103.40,741.6,0.1791,0.5249,0.5355,0.1741,0.3985,0.12440
6,M,18.25,19.98,119.60,1040.0,0.09463,0.10900,0.11270,0.07400,0.1794,...,22.88,27.66,153.20,1606.0,0.1442,0.2576,0.3784,0.1932,0.3063,0.08368
7,M,13.71,20.83,90.20,577.9,0.11890,0.16450,0.09366,0.05985,0.2196,...,17.06,28.14,110.60,897.0,0.1654,0.3682,0.2678,0.1556,0.3196,0.11510
8,M,13.00,21.82,87.50,519.8,0.12730,0.19320,0.18590,0.09353,0.2350,...,15.49,30.73,106.20,739.3,0.1703,0.5401,0.5390,0.2060,0.4378,0.10720
9,M,12.46,24.04,83.97,475.9,0.11860,0.23960,0.22730,0.08543,0.2030,...,15.09,40.68,97.65,711.4,0.1853,1.0580,1.1050,0.2210,0.4366,0.20750


In [3]:
df.iloc[:,0].value_counts()

B    357
M    212
Name: 1, dtype: int64

In [4]:
df.isna().any().any()

False

## (b) Monte-Carlo Simulation: Repeat the following procedures for supervised, un- supervised, and semi-supervised learning M = 30 times, and use randomly se- lected train and test data (make sure you use 20% of both the positve and nega- tive classes as the test set). Then compare the average scores (accuracy, precision, recall, F1-score, and AUC) that you obtain from each algorithm.

### i. Supervised Learning: Train an L1-penalized SVM to classify the data. Use 5 fold cross validation to choose the penalty parameter. Use normalized data. Report the average accuracy, precision, recall, F1-score, and AUC, for both training and test sets over your M runs. Plot the ROC and report the confusion matrix for training and testing in one of the runs.

In [5]:
X, y = df.iloc[:,1:], df.iloc[:,0]
LE = LabelEncoder()
y = LE.fit_transform(y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

In [6]:
Counter(y_test)

Counter({0: 72, 1: 42})

In [7]:
X_test.head(10)

,2,3,4,5,6,7,8,9,10,11,...,22,23,24,25,26,27,28,29,30,31
557,9.423,27.88,59.26,271.3,0.08123,0.04971,0.00000,0.00000,0.1742,0.06059,...,10.490,34.24,66.50,330.6,0.1073,0.07158,0.0000,0.00000,0.2475,0.06969
111,12.630,20.76,82.15,480.4,0.09933,0.12090,0.10650,0.06021,0.1735,0.07070,...,13.330,25.47,89.00,527.4,0.1287,0.22500,0.2216,0.11050,0.2226,0.08486
538,7.729,25.49,47.98,178.8,0.08098,0.04878,0.00000,0.00000,0.1870,0.07285,...,9.077,30.92,57.17,248.0,0.1256,0.08340,0.0000,0.00000,0.3058,0.09938
96,12.180,17.84,77.79,451.1,0.10450,0.07057,0.02490,0.02941,0.1900,0.06635,...,12.830,20.92,82.14,495.2,0.1140,0.09358,0.0498,0.05882,0.2227,0.07376
465,13.240,20.13,86.87,542.9,0.08284,0.12230,0.10100,0.02833,0.1601,0.06432,...,15.440,25.50,115.00,733.5,0.1201,0.56460,0.6556,0.13570,0.2845,0.12490
449,21.100,20.52,138.10,1384.0,0.09684,0.11750,0.15720,0.11550,0.1554,0.05661,...,25.680,32.07,168.20,2022.0,0.1368,0.31010,0.4399,0.22800,0.2268,0.07425
272,21.750,20.99,147.30,1491.0,0.09401,0.19610,0.21950,0.10880,0.1721,0.06194,...,28.190,28.18,195.90,2384.0,0.1272,0.47250,0.5807,0.18410,0.2833,0.08858
480,12.160,18.03,78.29,455.3,0.09087,0.07838,0.02916,0.01527,0.1464,0.06284,...,13.340,27.87,88.83,547.4,0.1208,0.22790,0.1620,0.05690,0.2406,0.07729
312,12.760,13.37,82.29,504.1,0.08794,0.07948,0.04052,0.02548,0.1601,0.06140,...,14.190,16.40,92.04,618.8,0.1194,0.22080,0.1769,0.08411,0.2564,0.08253
351,15.750,19.22,107.10,758.6,0.12430,0.23640,0.29140,0.12420,0.2375,0.07603,...,17.360,24.17,119.40,915.3,0.1550,0.50460,0.6872,0.21350,0.4245,0.10500


In [8]:
steps = [('scaler', MinMaxScaler()),
         ('SVM',  LinearSVC(penalty='l1', dual=False, tol=1e-2))]
pipeline = Pipeline(steps)

parameters = {'SVM__C': np.logspace(-10, 10, 20)}

cv = GridSearchCV(pipeline, param_grid = parameters, cv = 5, scoring='accuracy',return_train_score=True, refit=True, n_jobs=-1)
cv.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scaler', MinMaxScaler()),
                                       ('SVM',
                                        LinearSVC(dual=False, penalty='l1',
                                                  tol=0.01))]),
             n_jobs=-1,
             param_grid={'SVM__C': array([1.00000000e-10, 1.12883789e-09, 1.27427499e-08, 1.43844989e-07,
       1.62377674e-06, 1.83298071e-05, 2.06913808e-04, 2.33572147e-03,
       2.63665090e-02, 2.97635144e-01, 3.35981829e+00, 3.79269019e+01,
       4.28133240e+02, 4.83293024e+03, 5.45559478e+04, 6.15848211e+05,
       6.95192796e+06, 7.84759970e+07, 8.85866790e+08, 1.00000000e+10])},
             return_train_score=True, scoring='accuracy')

In [9]:
best_model = cv.best_estimator_
print(cv.best_score_)
print(cv.best_params_)

0.9736263736263737
{'SVM__C': 3.359818286283774}


In [11]:
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
ypreds = best_model.predict(X_train_scaled)
yhat = best_model.predict(X_test_scaled)

print("Train Accuracy", accuracy_score(ypreds, y_train))
print(Counter(ypreds))
print(Counter(y_train))
print("Train Precision", precision_score(ypreds, y_train))
print("Train Recall Score", recall_score(ypreds, y_train))
print("Train F1 Score", f1_score(ypreds, y_train))
roc_auc_c



Train Accuracy 0.7648351648351648
Counter({0: 392, 1: 63})
Counter({0: 285, 1: 170})
Train Precision 0.37058823529411766
Train Recall Score 1.0
Train F1 Score 0.5407725321888412
